In [1]:
import pandas as pd
import json
import os
import numpy as np
import pickle
import seaborn as sns
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torchmetrics import F1Score
from torch.utils.data import Dataset, DataLoader

%matplotlib inline

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
# Класс - модуль, реализующий архитектуру. Можно было бы использовать nn.Sequential
class NeuralNetModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetModel, self).__init__()

        # Первый полносвязный слой
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim, device=device) 

        # Нелинейная функция
        self.activation = torch.nn.ReLU()
        
        # Полносвязный слой
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim, device=device)  
        
    def forward(self, x):
        # Полносвязный слой
        out = self.fc1(x)

        # Нелинейная функция
        out = self.activation(out)

        # Полносвязный слой
        out = self.fc2(out)
        
        return out

In [6]:
model = NeuralNetModel(5, 10, 1)

In [27]:
weight = torch.tensor([[1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5],
                      [1, 2, 3, 4, 5]])
weight = weight.type(torch.float32)

In [28]:
weight.shape

torch.Size([10, 5])

In [29]:
weight

tensor([[1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.]])

In [31]:
model.fc1.weight

Parameter containing:
tensor([[ 0.4156,  0.4188,  0.4409, -0.2223, -0.0092],
        [ 0.4357, -0.2379, -0.3851,  0.0518, -0.1201],
        [ 0.0167,  0.2359, -0.2200, -0.0249,  0.2474],
        [-0.1188, -0.3928, -0.0895,  0.2007, -0.3319],
        [-0.2384,  0.4274, -0.3406,  0.2797,  0.0418],
        [ 0.1163,  0.2719,  0.4238, -0.0488,  0.1991],
        [ 0.3628,  0.3552, -0.2150,  0.3740,  0.3271],
        [ 0.2066,  0.2653,  0.0891,  0.2110,  0.1955],
        [ 0.2879, -0.2189,  0.3884, -0.2384, -0.2715],
        [ 0.1236, -0.1306, -0.0804, -0.3025,  0.1951]], device='cuda:0',
       requires_grad=True)

In [33]:
print(model)

NeuralNetModel(
  (fc1): Linear(in_features=5, out_features=10, bias=True)
  (activation): ReLU()
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)


In [34]:
model.named_parameters

<bound method Module.named_parameters of NeuralNetModel(
  (fc1): Linear(in_features=5, out_features=10, bias=True)
  (activation): ReLU()
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)>

In [3]:
df = pd.read_json('../df.json')